# From CM to trial results

## Introduction

Goal of this cookbook is to illustrate how one can post the material needed for a trial, run it and visualize the results.

Steps: 
1. Post a Computational Model
2. Create a Virtual Population (Vpop) Design
3. Generate a Vpop from the Vpop Design
4. Post a Protocol
5. Post a Data Table
6. Post a Trial
7. Run and monitor a Trial
8. Visualize the trial results

Linked resources: 
- [Folder on jinko](https://jinko.ai/project/e0fbb5bb-8929-439a-bad6-9e12d19d9ae4).

**Note on known bug currently being solved:**
 - Step 1 : the computational model cannot for now contain extra options. This forces it be named "Untitled" and to be located at the source of the jinko.ai project

In [ ]:
# Jinko specifics imports & initialization
# Please fold this section and do not change

import sys

sys.path.insert(0, "../lib")
import jinko_helpers as jinko

# Connect to Jinko (see README.md for more options)

jinko.initialize()

In [ ]:
# Cookbook specifics imports

import io
import json
import os
import pandas as pd
import plotly.express as px
import zipfile

# Cookbook specifics constants:
# put here the constants that are specific to your cookbook like
# the reference to the Jinko items, the name of the model, etc.

# folder_id can be retrieved in the url, pattern is `https://jinko.ai/project/<project_id>?labels=<folder_id>`
folder_id = "5f573c8f-3f48-4c48-8257-f884837e5605"

resources_dir = os.path.normpath("resources/run_a_trial")

model_file = os.path.join(resources_dir, "computational_model.json")
solving_options_file = os.path.join(resources_dir, "solving_options.json")
vpop_file = os.path.join(resources_dir, "vpop.csv")
protocol_file = os.path.join(resources_dir, "protocol.json")
data_table_file = os.path.join(resources_dir, "data_table.csv")

# Step 1: Post a Computational Model

In [ ]:
# Load the model
with open(model_file, "r") as f:
    model = json.load(f)

# Load the solving options
with open(solving_options_file, "r") as f:
    solving_options = json.load(f)

# Post the model with its options
# https://doc.jinko.ai/api/#/paths/core-v2-model_manager-jinko_model/post
response = jinko.makeRequest(
    path="/core/v2/model_manager/jinko_model",
    method="POST",
    json={"model": model, "solvingOptions": solving_options},
)

response_json = response.json()

model_core_item_id = response_json.get("coreItemId")
model_snapshot_id = response_json.get("snapshotId")

# Get the URL of the resource
jinko.getProjectItemUrlByCoreItemId(model_core_item_id)

Here since Cm is missing a X-jinko-project-item-folder-ids header, you can move the CM in the correct folder (known bug being solved by the team)

# Step 2: Create a Vpop Design

### Get model descriptors


In [ ]:
# get model baseline descriptors
# https://doc.jinko.ai/api/#/paths/core-v2-model_manager-jinko_model-jinkoModelId--snapshots--jinkoModelIdSnapshot--baseline_descriptors/get
response = jinko.makeRequest(
    path=f"/core/v2/model_manager/jinko_model/{model_core_item_id}/snapshots/{model_snapshot_id}/baseline_descriptors",
)

response_json = response.json()

numeric_descriptors = response_json["numericDescriptors"]

# build the default marginal distributions that will be used to create the vpop design
# we only select descriptors that are PatientDescriptorKnown, PatientDescriptorUnknown or PatientDescriptorPartiallyKnown
default_marginal_distributions = [
    {
        "distribution": {
            "highBound": descriptor["distribution"]["highBound"],
            "lowBound": descriptor["distribution"]["lowBound"],
            "tag": descriptor["distribution"]["tag"],
        },
        "id": descriptor["id"],
    }
    for descriptor in numeric_descriptors
    if any(
        tag in descriptor["inputTag"]
        for tag in [
            "PatientDescriptorKnown",
            "PatientDescriptorUnknown",
            "PatientDescriptorPartiallyKnown",
        ]
    )
]

# Creating a formatted message with the IDs from marginal distributions
ids_output = "IDs present in the Marginal Distributions:\n" + "\n".join(
    [distribution["id"] for distribution in default_marginal_distributions]
)

default_marginal_distributions

### Create a new list with the updated distributions

In [ ]:
# Define a dictionary for distribution settings
distribution_settings = {
    "initialTumorBurden": {"mean": 1.8, "stdev": 0.08, "base": 10, "tag": "LogNormal"},
    "kccCancerCell": {"mean": 12, "stdev": 0.5, "base": 10, "tag": "LogNormal"},
    "kGrowthCancerCell": {"mean": -3, "stdev": 0.05, "base": 10, "tag": "LogNormal"},
    "vmaxCancerCellDeath": {"mean": -1, "stdev": 0.05, "base": 10, "tag": "LogNormal"},
    "ec50Drug": {"mean": -3.5, "stdev": 0.05, "base": 10, "tag": "LogNormal"},
}

# Refactor the list comprehension using the dictionary
updated_marginal_distributions = [
    {
        "id": element["id"],
        "distribution": distribution_settings.get(
            element["id"],
            element[
                "distribution"
            ],  # Default to the existing distribution if id is not found
        ),
    }
    for element in default_marginal_distributions
]

### Post the vpop design

In [ ]:
# https://doc.jinko.ai/api/#/paths/core-v2-vpop_manager-vpop_generator/post
response = jinko.makeRequest(
    path="/core/v2/vpop_manager/vpop_generator",
    method="POST",
    json={
        "contents": {
            "computationalModelId": {
                "coreItemId": model_core_item_id,
                "snapshotId": model_snapshot_id,
            },
            "correlations": [],
            "marginalCategoricals": [],
            "marginalDistributions": updated_marginal_distributions,
        },
        "tag": "VpopGeneratorFromDesign",
    },
    options={
        "name": "vpop design for simple tumor model",
        "folder_id": folder_id,
    },
)

# Pretty print the response
response_json = response.json()
vpop_generator_core_item_id = response_json.get("coreItemId")
vpop_generator_snapshot_id = response_json.get("snapshotId")

# Get the URL of the resource
jinko.getProjectItemUrlByCoreItemId(vpop_generator_core_item_id)

# Step 3: Generate a Vpop from the Vpop design

In [ ]:
# https://doc.jinko.ai/api/#/paths/core-v2-vpop_manager-vpop_generator-vpopGeneratorId--snapshots--vpopGeneratorIdSnapshot--vpop/post

response = jinko.makeRequest(
    path=f"/core/v2/vpop_manager/vpop_generator/{vpop_generator_core_item_id}/snapshots/{vpop_generator_snapshot_id}/vpop",
    method="POST",
    json={
        "contents": {
            "computationalModelId": {
                "coreItemId": model_core_item_id,
                "snapshotId": model_snapshot_id,
            },
            "size": 10,  # vpop has 10 patients
        },
        "tag": "VpopGeneratorOptionsForVpopDesign",
    },
    options={
        "name": "vpop for simple tumor model",
        "folder_id": folder_id,
    },
)

response_json = response.json()
print(response_json)
vpop_core_item_id = response_json["coreItemId"]
vpop_snapshot_id = response_json["snapshotId"]

# Step 3 bis - not mandatory: Directly post a csv vpop

In [ ]:
with open(vpop_file, "r") as file:
    vpop = file.read()

# https://doc.jinko.ai/api/#/paths/core-v2-vpop_manager-vpop/post

response = jinko.makeRequest(
    path=f"/core/v2/vpop_manager/vpop",
    method="POST",
    csv_data=vpop,
    options={
        "name": "vpop for simple tumor model",
        "folder_id": folder_id,
    },
)
response_json = response.json()

vpop_bis_core_item_id = response_json["coreItemId"]
vpop_bis_snapshot_id = response_json["snapshotId"]

# Step 4 : Post a Protocol

In [ ]:
# Load the protocol
with open(protocol_file, "r") as f:
    protocol = json.load(f)

# Post the protocol
# https://doc.jinko.ai/api/#/paths/core-v2-scenario_manager-protocol_design/post
response = jinko.makeRequest(
    path="/core/v2/scenario_manager/protocol_design",
    method="POST",
    json=protocol,
    options={
        "name": "protocol for simple tumor model",
        "folder_id": folder_id,
    },
)

response_json = response.json()

protocol_core_item_id = response_json.get("coreItemId")
protocol_snapshot_id = response_json.get("snapshotId")

# Get the URL of the resource
jinko.getProjectItemUrlByCoreItemId(protocol_core_item_id)

# Step 5: Post a Data Table

In [ ]:
# the source data table is a CSV file, we convert it to SQLite and encode it in base64

encoded_data_table = jinko.dataTableToSQLite(data_table_file)

# Step 3: Post the data table
# https://doc.jinko.ai/api/#/paths/core-v2-data_table_manager-data_table/post
response = jinko.makeRequest(
    path="/core/v2/data_table_manager/data_table",
    method="POST",
    json={
        "mappings": [],
        "rawData": encoded_data_table,
    },
    options={
        "name": "data table for simple tumor model",
        "folder_id": folder_id,
    },
)

# Pretty print the response
response_json = response.json()

data_table_core_item_id = response_json.get("coreItemId")
data_table_snapshot_id = response_json.get("snapshotId")

# Get the URL of the resource
jinko.getProjectItemUrlByCoreItemId(data_table_core_item_id)

# Step 6: Post a Trial

In [ ]:
# https://doc.jinko.ai/api/#/paths/core-v2-trial_manager-trial/post

# Define the data payload
trial_data = {
    "computationalModelId": {
        "coreItemId": model_core_item_id,
        "snapshotId": model_snapshot_id,
    },
    "protocolDesignId": {
        "coreItemId": protocol_core_item_id,
        "snapshotId": protocol_snapshot_id,
    },
    "vpopId": {"coreItemId": vpop_core_item_id, "snapshotId": vpop_snapshot_id},
    "dataTableDesigns": [
        {
            "dataTableId": {
                "coreItemId": data_table_core_item_id,
                "snapshotId": data_table_snapshot_id,
            },
            "options": {
                "logTransformWideBounds": [],
                "label": "data_table_simple_tumor",
            },
            "include": True,
        }
    ],
}

response = jinko.makeRequest(
    path="/core/v2/trial_manager/trial",
    method="POST",
    json=trial_data,
    options={
        "name": "trial for simple tumor model",
        "folder_id": folder_id,
    },
)

response_json = response.json()

trial_core_item_id = response_json.get("coreItemId")
trial_snapshot_id = response_json.get("snapshotId")

trial_id = {"coreItemId": trial_core_item_id, "snapshotId": trial_snapshot_id}

# Get the URL of the resource
jinko.getProjectItemUrlByCoreItemId(trial_core_item_id)

# Step 7 : Run and monitor a trial


### Run the trial

In [ ]:
# https://doc.jinko.ai/api/#/paths/core-v2-trial_manager-trial-trialId--snapshots--trialIdSnapshot--run/post
response = jinko.makeRequest(
    path=f"/core/v2/trial_manager/trial/{trial_core_item_id}/snapshots/{trial_snapshot_id}/run",
    method="POST",
)

### get trial status


In [ ]:
# https://doc.jinko.ai/api/#/paths/core-v2-trial_manager-trial-trialId--snapshots--trialIdSnapshot--status/get
response = jinko.makeRequest(
    path=f"/core/v2/trial_manager/trial/{trial_core_item_id}/snapshots/{trial_snapshot_id}/status"
)
response_json = response.json()
# Extract 'perArmSummary' data
per_arm_data = response_json.get("perArmSummary", {})

# Check if the job is running and print the appropriate message
if response_json.get("isRunning", False):
    print("Job is running.")
else:
    print("Job succeeded.")

# Create a DataFrame without hardcoding column names
if per_arm_data:
    per_arm_summary = pd.DataFrame.from_dict(per_arm_data, orient="index")
    per_arm_summary.reset_index(inplace=True)
    per_arm_summary.rename(columns={"index": "Arm"}, inplace=True)
    display(per_arm_summary)
else:
    print("No 'perArmSummary' data found in the response.")

# Step 8 : Visualize the trial results 

In [ ]:
# Retrieve time series ids (https://doc.jinko.ai/api/#/paths/core-v2-trial_manager-trial-trialId--snapshots--trialIdSnapshot--output_ids/get)

response = jinko.makeRequest(
    "/core/v2/trial_manager/trial/%s/snapshots/%s/output_ids"
    % (trial_core_item_id, trial_snapshot_id),
    method="GET",
)
responseSummary = json.loads(response.content.decode("utf-8"))
print("Available time series:\n", responseSummary, "\n")

In [ ]:
# Retrieve time series (https://doc.jinko.ai/api/#/paths/core-v2-result_manager-timeseries_summary/post)

# replace here by the time series ids list you want
idsForTimeSeries = ["tumorBurden"]

try:
    print("Retrieving time series data...")
    response = jinko.makeRequest(
        "/core/v2/result_manager/timeseries_summary",
        method="POST",
        json={
            "select": idsForTimeSeries,
            "trialId": trial_id,
        },
    )
    if response.status_code == 200:
        print("Time series data retrieved successfully.")
        archive = zipfile.ZipFile(io.BytesIO(response.content))
        filename = archive.namelist()[0]
        print(f"Extracted time series file: {filename}")
        csvTimeSeries = archive.read(filename).decode("utf-8")
    else:
        print(
            f"Failed to retrieve time series data: {response.status_code} - {response.reason}"
        )
        response.raise_for_status()
except Exception as e:
    print(f"Error during time series retrieval or processing: {e}")
    raise

In [ ]:
dfTimeSeries = pd.read_csv(io.StringIO(csvTimeSeries))
display(dfTimeSeries.head(5))

# Extract unique patient IDs
unique_patient_ids = dfTimeSeries["Patient Id"].unique().tolist()

# Display unique patient IDs
print(unique_patient_ids)

In [ ]:
# Filter data for the first patient
patient_data = dfTimeSeries[dfTimeSeries["Patient Id"] == unique_patient_ids[0]]

# Plot using Plotly
fig = px.line(
    patient_data,
    x="Time",
    y="Value",
    color="Arm",
    title="Time Series of Tumor Burden",
    labels={"Time": "Time (seconds)", "Value": "Tumor Burden Value"},
    markers=True,
)

fig.show()